<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/AI_at_Scale/Assignments/Assignment2/Assignment2U.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark pandas

Use the Bank Marketing dataset (saved in drive) to build a binary
classifier that predicts whether a customer will subscribe to a term deposit
using Spark MLlib.


Do the following tasks on the dataset:


Problem 1 : Data Understanding (Marks : 10)

In [10]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import regexp_replace, col, countDistinct, col, sum
from pyspark.sql.types import StringType, DoubleType, IntegerType ,StructType,StructField
from pyspark.ml.feature import Imputer, VectorAssembler,StandardScaler
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import time
from pyspark.ml.regression import RandomForestRegressor, DecisionTreeRegressor
from pyspark.ml import PipelineModel

In [16]:
import csv

def sniff_delimiter(file_path):
    with open(file_path, 'r') as f:
        # Read a sample of the file
        sample = f.read(4096)
        # Use the sniffer to guess the delimiter
        sniffer = csv.Sniffer()
        try:
            dialect = sniffer.sniff(sample)
            return dialect.delimiter
        except csv.Error:
            return "Could not determine delimiter automatically. Please inspect the file manually."

file_path = 'bank-full.csv'
delimiter = sniff_delimiter(file_path)
print(f"Detected delimiter: {delimiter}")

Detected delimiter: ;


In [11]:
customSchema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", DoubleType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),
    StructField("contact", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("duration", IntegerType(), True),
    StructField("campaign", IntegerType(), True),
    StructField("pdays", IntegerType(), True),
    StructField("previous", IntegerType(), True),
    StructField("poutcome", StringType(), True),
    StructField("y", StringType(), True)

])

In [14]:
# Load the dataset and print schema

spark_c = SparkSession.builder \
    .appName("Assignment2") \
    .config("spark.executor.cores","2") \
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

data = spark_c.read.csv("bank-full.csv", header=True, schema=customSchema, sep=';')
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [15]:
data.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no| 2143.0|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|   29.0|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|    2.0|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no| 1506.0|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|    1.0|     no|  no|unknown|  5|  may

In [17]:
# Count how many customers subscribed (y = yes) vs not.

subscription_counts = data.groupBy("y").count()
subscription_counts.show()


+---+-----+
|  y|count|
+---+-----+
| no|39922|
|yes| 5289|
+---+-----+



In [ ]:
# Print the distinct values of job and education and count of each value

